In [4]:
from PIL import Image,ImageDraw,ImageOps
import face_recognition
import os
import pickle
# from sklearn import neighbors
# from face_recognition.face_recognition_cli import image_files_in_folder
import numpy as np
from dataclasses import dataclass

# Load the jpg file into a numpy array
# image = face_recognition.load_image_file("photos/DSC05406.JPG")

### prepare jpg files

In [5]:
import glob
photo_path = glob.glob("./photos/*.JPG")

def rotate_if_landscape(photo_path):
    # Open the image
    
    try:
        im=Image.open(photo_path)
        im=ImageOps.exif_transpose(im)
    except:
        im=Image.open(photo_path)

    im.save(photo_path)

# Example usage:
        
for img in photo_path:
    # rotate_if_landscape(img)
    print(img)

./photos/DSC05301.JPG
./photos/DSC05300.JPG
./photos/DSC05299.JPG
./photos/DSC05298.JPG
./photos/DSC05297.JPG
./photos/DSC05296.JPG
./photos/DSC05295.JPG
./photos/DSC05294.JPG
./photos/DSC05293.JPG
./photos/DSC05292.JPG
./photos/DSC05291.JPG
./photos/DSC05290.JPG
./photos/DSC05289.JPG
./photos/DSC05288.JPG


### read and store pictures

In [6]:
@dataclass
class read_pic_config:
    pic_path: str = None  # Default value set to None
    artifact_path: str= "./artifacts/lib_pic.pkl"
    def __post_init__(self):
        if self.pic_path is None:
            self.pic_path = glob.glob("./photos/*.JPG")

class read_pic:
    def __init__(self) -> None:
        self.config=read_pic_config()
        self.photo_path=self.config.pic_path
        self.artifact_path=self.config.artifact_path
        self.save_image()
    
    def save_image(self):
        pic_id=0
        self.pic_dict={}
        for path in self.photo_path:
            self.pic_dict[pic_id]=path
            pic_id=pic_id+1
            # im=Image.open(photo_path)
            # try:
            #     pic_dict[pic_id]=ImageOps.exif_transpose(im)
            # except:
            #     pic_dict[pic_id]=im
        
        os.makedirs(os.path.dirname(self.artifact_path),exist_ok=True)
        with open(self.artifact_path, 'wb') as f:
            pickle.dump(self.pic_dict, f)


read_pic()


In [2]:
with open(read_pic_config.artifact_path, 'rb') as f:
    pics = pickle.load(f)

pics
# x=Image.open(pics[1])
# x.show()

NameError: name 'read_pic_config' is not defined

### encode pics

In [7]:
from collections import defaultdict
@dataclass
class batch_embedding_config:
    encoding_path: str = './artifacts/lib_encode.pkl'  # Default value set to None
    face_path: str = './artifacts/lib_face.pkl' 
class batch_embedding:
    def __init__(self):
        self.faces=[]
        pic_path=read_pic_config().artifact_path
        self.encoding_path=batch_embedding_config().encoding_path
        self.face_path=batch_embedding_config().face_path
        with open(pic_path, 'rb') as f:
            self.pics_dict = pickle.load(f)
        
        self.encodings=np.empty((0, 129))
        self.encode()
        

    def encode(self):
        for pic_id,pic in self.pics_dict.items():
            X_img = face_recognition.load_image_file(pic)
            X_face_locations = face_recognition.face_locations(X_img)
            if len(X_face_locations)==0:
                continue

            faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)
            num_rows = len(faces_encodings)
            index_array=np.full((num_rows,1),pic_id)  
            faces_encodings_with_index = np.hstack((index_array, faces_encodings))

            for face_location in X_face_locations:
                # Print the location of each face in this image
                top, right, bottom, left = face_location
                face_image = X_img[top:bottom, left:right]
                pil_image = Image.fromarray(face_image)
                self.faces.append(pil_image)

            self.encodings=np.vstack((self.encodings, faces_encodings_with_index))

        with open(self.encoding_path, 'wb') as f:
            pickle.dump(self.encodings, f)
        with open(self.face_path, 'wb') as f:
            pickle.dump(self.faces, f)
        

batch_embedding()


In [8]:
with open(batch_embedding_config.encoding_path, 'rb') as f:
    ecode = pickle.load(f)
with open(batch_embedding_config.face_path, 'rb') as f:
    face = pickle.load(f)

len(ecode)

# k=6
# for i  in face[k]:
#     i.show()
# img=Image.open(pics_dict[k])
# img.show()


45

### group 

In [8]:
@dataclass
class group_pic_config:
    group_path: str = './artifacts/group_id.pkl'  
    group_face_path: str = './artifacts/group_face.pkl'  
    group_encode: str = './artifacts/group_encode.pkl'  

class group_pic:
    def __init__(self) -> None:
        config=group_pic_config
        self.group_path=config.group_path
        self.group_face_path=config.group_face_path
        self.group_encode= config.group_encode

        encoding_path=batch_embedding_config.encoding_path
        face_lib_path=batch_embedding_config.face_path

        
        with open(encoding_path, 'rb') as f:
            self.encoding = pickle.load(f)
        with open(face_lib_path, 'rb') as f:
            self.face = pickle.load(f)

        self.store={}
        self.store_id={}
        self.faces={}

        self.group()
 
    def group(self):
        for i,k in enumerate(self.encoding):
            action=False
            curr_image_cluster_id=None   

            img_id:int=k[0]
            img= k[1:]
                   
            for cluster_id,cluster_embeddings in self.store.items():
                results=face_recognition.compare_faces(cluster_embeddings,img,tolerance=0.4)
                # results=face_recognition.compare_faces([np.mean(cluster_embeddings,axis=0)],img,tolerance=0.3)
                # print("results %s %s" % (results, cluster_id))

                if all(results):
                    curr_image_cluster_id = cluster_id
                    self.store.get(cluster_id).append(img)
                    self.store_id.get(cluster_id).append(img_id)
                    self.faces.get(cluster_id).append(self.face[i])
                    action=True

            if not action:
                curr_image_cluster_id="person_%s" % (len(self.store.keys()) + 1)
                self.store[curr_image_cluster_id]=[img]
                self.store_id[curr_image_cluster_id]=[img_id]
                self.faces[curr_image_cluster_id]=[self.face[i]]
        
        with open(self.group_path, 'wb') as f:
            pickle.dump(self.store_id, f)    
        with open(self.group_face_path, 'wb') as f:
            pickle.dump(self.faces, f) 
        with open(self.group_encode, 'wb') as f:
            gp_encode={}
            for cluster_id,cluster_embeddings in self.store.items():
                gp_encode[cluster_id]=np.mean(cluster_embeddings,axis=0)
            
            
            pickle.dump(gp_encode, f) 

a=group_pic()

In [19]:
pic_path=read_pic_config().artifact_path
with open(pic_path, 'rb') as f:
    pics_dict = pickle.load(f)

with open(group_pic_config.group_path, 'rb') as f:
    group = pickle.load(f)

with open(group_pic_config.group_face_path, 'rb') as f:
    groupface = pickle.load(f)

with open(group_pic_config.group_encode, 'rb') as f:
    groupencode = pickle.load(f)

# for i,j in groupface.items():
#     # img=Image.open(pics_dict[int(i)])
#     j[0].show()


from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
# (groupencode.values())
vectors = list(groupencode.values())
data = np.array(vectors)
similarity_matrix = pairwise_distances(data)
print(similarity_matrix)


[[0.         0.26571771 0.48717233 0.30418473 0.45711627 0.50131803
  0.47214846 0.46467763 0.66508356 0.29435884]
 [0.26571771 0.         0.54302577 0.11214331 0.51283403 0.57362856
  0.50680518 0.53333651 0.68298367 0.34058242]
 [0.48717233 0.54302577 0.         0.55344314 0.17809393 0.46177329
  0.20094128 0.27566293 0.48449196 0.59698495]
 [0.30418473 0.11214331 0.55344314 0.         0.52202922 0.57018973
  0.52589282 0.53364562 0.68180394 0.34847656]
 [0.45711627 0.51283403 0.17809393 0.52202922 0.         0.43979388
  0.23567    0.26081552 0.47952004 0.55413044]
 [0.50131803 0.57362856 0.46177329 0.57018973 0.43979388 0.
  0.49966742 0.42198052 0.59404954 0.57697821]
 [0.47214846 0.50680518 0.20094128 0.52589282 0.23567    0.49966742
  0.         0.32628855 0.49579355 0.56194766]
 [0.46467763 0.53333651 0.27566293 0.53364562 0.26081552 0.42198052
  0.32628855 0.         0.44415816 0.5761615 ]
 [0.66508356 0.68298367 0.48449196 0.68180394 0.47952004 0.59404954
  0.49579355 0.44415

### save grouped pics

In [9]:
from shutil import copyfile
from pathlib import Path
@dataclass
class save_pic_config:
    pic_path: str = './results'  # Default value set to None
class save_pic:

    def __init__(self) -> None:

        with open(group_pic_config.group_face_path, 'rb') as f:
            self.groupface = pickle.load(f)
        self.curr_cluster = os.path.join(save_pic_config.pic_path)

        with open(group_pic_config.group_path, 'rb') as f:
            self.group = pickle.load(f)

        with open(read_pic_config.artifact_path, 'rb') as f:
            self.pics_dict = pickle.load(f)

        self.save_grouped_face()

    def save_grouped_pic(self):
        
        for key, pic in self.group.items():
            curr_cluster = os.path.join(self.curr_cluster, key)
            curr_cluster_dir = Path(curr_cluster)
            curr_cluster_dir.mkdir(parents=True, exist_ok=True)
            

            img=Image.open(self.pics_dict[int()])

            i=1
            for im in pic:
                img=Image.open(self.pics_dict[int(im)])

                photo_path=os.path.join(curr_cluster,str(i)+'.jpg')
                img.save(photo_path)
                # print(photo_path)
                i=i+1

    def save_grouped_face(self):
        
        for key, pic in self.groupface.items():
            curr_cluster = os.path.join(self.curr_cluster, key)
            curr_cluster_dir = Path(curr_cluster)
            curr_cluster_dir.mkdir(parents=True, exist_ok=True)
            
            i=1
            for im in pic:
                # im.show()
                # im.save(photo_path)
                photo_path=os.path.join(curr_cluster,str(i)+'.jpg')
                im.save(photo_path)
                # print(photo_path)
                i=i+1
            

save_pic()

### encode some photos

In [ ]:
class batch_embedding:
    def __init__(self, photo_path):
        self.photo_paths=photo_path
        self.photos=[]
        self.encodings=np.empty((0, 128))
        self.store={}
        self.store_img={}

    def encode(self):
        for photo_path in self.photo_paths:
            X_img = face_recognition.load_image_file(photo_path)
            X_face_locations = face_recognition.face_locations(X_img)
            if len(X_face_locations)==0:
                continue
            faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)
            # print('faces encodings size is {}'.format(len(faces_encodings)))
            # print('total encodings size is {}'.format(len(self.encodings)))
            self.encodings=np.vstack((self.encodings, faces_encodings))

            for face_location in X_face_locations:

                # Print the location of each face in this image
                top, right, bottom, left = face_location

                face_image = X_img[top:bottom, left:right]
                pil_image = Image.fromarray(face_image)
                
                self.photos.append(pil_image)
                # pil_image.show()
        with open('encodings.pkl', 'wb') as f:
            pickle.dump(zip(self.encodings,self.photos), f)
        return self.encodings, self.photos
    
    # def save(self,encodings):
    #     with open('encodings.pkl', 'wb') as f:
    #         pickle.dump(encodings, f)

    def group(self):
        for fe,img in zip(self.encodings,self.photos):
            action=False
            curr_image_cluster_id=None    
                   
            for cluster_id,cluster_embeddings in self.store.items():
                results=face_recognition.compare_faces([np.mean(cluster_embeddings,axis=0)],fe,tolerance=0.4)
                print("results %s %s" % (results, cluster_id))

                if all(results):
                    curr_image_cluster_id = cluster_id
                    self.store.get(cluster_id).append(fe)
                    self.store_img.get(cluster_id).append(img)
                    action=True

            if not action:
                curr_image_cluster_id="cluster_%s" % (len(self.store.keys()) + 1)
                self.store[curr_image_cluster_id]=[fe]
                self.store_img[curr_image_cluster_id]=[img]
        return self.store, self.store_img
                    



In [ ]:
encode=batch_embedding(photo_path)
encoding,photos=encode.encode()


In [ ]:
num_rows = encoding.shape[0]
index_array=np.full((num_rows,1),2)
index_array
array_with_index = np.hstack((index_array, encoding))
len(array_with_index[0])

In [ ]:
# a=face_recognition.compare_faces(encoding,encoding[1],tolerance=0.4)
# print(a)
# np.sum(a) > len(a) / 2
len(np.mean(encoding,axis=0))

In [ ]:
for i in grouped_img['cluster_4']:
        i.show()

In [ ]:
# Find all the faces in the image using the default HOG-based model.
# This method is fairly accurate, but not as accurate as the CNN model and not GPU accelerated.
# See also: find_faces_in_picture_cnn.py
face_locations = face_recognition.face_locations(image)

print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    
    pil_image.show()

## extract faces

In [ ]:
import numpy as np
faces_encodings = face_recognition.face_encodings(image, known_face_locations=face_locations)
temp=faces_encodings
faces_encodings.append(temp[:])
len(faces_encodings)

In [ ]:
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    
    X = []
    y = []

    # Loop through each person in the training set
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # Loop through each training image for the current person
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # Determine how many neighbors to use for weighting in the KNN classifier
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [ ]:
# Find all facial features in all the faces in the image
face_landmarks_list = face_recognition.face_landmarks(image)

print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

# Create a PIL imagedraw object so we can draw on the picture
pil_image = Image.fromarray(image)
d = ImageDraw.Draw(pil_image)

for face_landmarks in face_landmarks_list:

    # Print the location of each facial feature in this image
    for facial_feature in face_landmarks.keys():
        print("The {} in this face has the following points: {}".format(facial_feature, face_landmarks[facial_feature]))

    # Let's trace out each facial feature in the image with a line!
    for facial_feature in face_landmarks.keys():
        d.line(face_landmarks[facial_feature], width=5)

# Show the picture
pil_image.show()